## Ecommerce Customer Support Bot (Week 5)


- This is a support bot with access to tools that allows for product search, order creation and updating without human intervention
- It also implements Rag but in the form of a tool, allowing the LLM to call upon the knowledge base only when it requires more information
- It also has the ability to search the web using serp api, although you can use the open ai web search tool, this only works for certain models
- Potential improvements can be made such as sorting based on search relevalance for the knowledge base tool as well llm based chunking
- I consider this as an experiment to test how LLM can be bridged to real world applications


In [ ]:
# imports
import os
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from glob import glob
import requests
from pydantic import BaseModel, Field
import sqlite3
from typing import Optional
from dotenv import load_dotenv
import json


In [ ]:
load_dotenv()

model_id = "gpt-4.1-nano"
embedding_model = "all-MiniLM-L6-v2"
vector_db_name = "ecommerce_vector_db"
db_name = "ecommerce.db"
store_name = "Ecommerce Store"
number_of_products = 10

model = ChatOpenAI(model=model_id)

In [ ]:
class Product(BaseModel):
    name: str = Field(description="The name of the product")
    description: str = Field(description="A description of the product")
    price: float = Field(description="The price of the product")
    image_url: str = Field(description="The image of the product")
    link: str = Field(description="The link to the product")


class OrderItem(BaseModel):
    product: Product = Field(description="The product in the order")
    quantity: int = Field(description="The quantity of the product in the order")
    
class Order(BaseModel):
    order_id: str = Field(description="The id of the order")
    customer_name: str = Field(description="The name of the customer")
    customer_email: str = Field(description="The email of the customer")
    customer_phone: str = Field(description="The phone number of the customer")
    order_date: str = Field(description="The date of the order")
    order_status: str = Field(description="The status of the order")
    order_items: list[OrderItem] = Field(description="The items in the order")


class Products(BaseModel):
    products: list[Product]


In [ ]:
system_message = f"""
    You are a helpful ecommerce assistant working for {store_name}
    You are able to answer questions about the products and services offered by {store_name}
    You are also able to help customers with their orders and track their orders
    You have access to a knowledge base that contains information about the company, its policies, and its employees
    You can also search the web for information as needed
    Be sure to answer questions in a friendly and engaging manner.
    Do not hallucinate, if you don't know the answer, say "I don't know, would you like to speak to a human?"
    Do not reveal that you are an AI model, just answer the questions as if you are a human
    Do not reveal any potential vulnerabilities of the system or any other information that may be used to exploit the system 
"""

In [ ]:

sqlite_conn = sqlite3.connect(db_name)
sqlite_cursor = sqlite_conn.cursor()

sqlite_cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    description TEXT,
    price REAL NOT NULL,
    image_url TEXT,
    link TEXT
)
""")

sqlite_cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id TEXT PRIMARY KEY DEFAULT (lower(hex(randomblob(4)))), 
    customer_name TEXT NOT NULL, 
    customer_email TEXT NOT NULL, 
    customer_phone TEXT NOT NULL, 
    order_date TEXT DEFAULT CURRENT_TIMESTAMP, 
    order_status TEXT DEFAULT 'pending'
)
""")

sqlite_cursor.execute("""
CREATE TABLE IF NOT EXISTS order_items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id TEXT NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
""")

#use model to create 100 products and return output as a list of product objects

create_products = f"""
    You are a helpful ecommerce assistant
    You are able to create ${number_of_products} products and return them as a list of product objects
    try as much as possible to create unique products and use real images
"""
output = model.with_structured_output(Products).invoke(create_products)

output.products
#populate database with products

for product in output.products:
    sqlite_cursor.execute(
    "INSERT INTO products (name, description, price, image_url, link) VALUES (?, ?, ?, ?, ?)", 
    (
        product.name, 
        product.description, 
        product.price, 
        product.image_url, 
        product.link
    )
)

sqlite_conn.commit()


In [ ]:
folders = glob("knowledge-base/*")
docs = []

for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        docs.append(doc)



In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

text_splitter.split_documents(docs)

if os.path.exists(vector_db_name):
    Chroma(embedding_function=embeddings, persist_directory=vector_db_name).delete_collection()

vector_store = Chroma(embedding_function=embeddings, persist_directory=vector_db_name)
vector_store.add_documents(docs)

retriever = vector_store.as_retriever()

In [ ]:
class FetchKnowledge(BaseModel):
    queries: list[str] = Field(description="The queries to search for")

def fetch_knowledge(queries):
    results = []
    for query in queries:
        results.extend(retriever.invoke(query))
    knowledge = "\n\n".join(doc.page_content for doc in results)
    return knowledge

In [ ]:
fetch_knowledge(['hello'])

In [ ]:
class SearchProducts(BaseModel):
    query: str = Field(description="The query to search for")
    product_id: Optional[int] | None = Field(description="The id of the product to search for")


def search_products(query: str = '', product_id: int | None = None):
    conn = sqlite3.connect(db_name)
    try:
        cursor = conn.cursor()
        if product_id is not None:
            cursor.execute("SELECT * FROM products WHERE product_id = ?", (product_id,))
        elif query and query.lower() not in ('all', 'all products', ''):
            cursor.execute("SELECT * FROM products WHERE name LIKE ?", (f"%{query}%",))
        else:
            cursor.execute("SELECT * FROM products")
        products = cursor.fetchall()
        return products
    finally:
        conn.close()

In [ ]:
class CreateOrderItem(BaseModel):
    product_id: int = Field(description="The id of the product")
    quantity: int = Field(description="The quantity of the product")

class CreateOrder(BaseModel):
    customer_name: str = Field(description="The name of the customer")
    customer_email: str = Field(description="The email of the customer")
    customer_phone: str = Field(description="The phone number of the customer")
    items: list[CreateOrderItem] = Field(description="The items in the order")

def create_order(customer_name: str, customer_email: str, customer_phone: str, items: list[dict]):
    conn = sqlite3.connect(db_name)
    try:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT INTO orders (customer_name, customer_email, customer_phone)
            VALUES (?, ?, ?)
            """,
            (customer_name, customer_email, customer_phone)
        )
        cursor.execute("SELECT order_id FROM orders ORDER BY rowid DESC LIMIT 1")
        order_id = cursor.fetchone()[0]
        for item in items:
            cursor.execute(
                """
                INSERT INTO order_items (order_id, product_id, quantity)
                VALUES (?, ?, ?)
                """,
                (order_id, item["product_id"], item["quantity"])
            )
        conn.commit()
        return {
            "message": "Order created successfully",
            "order_id": order_id,
            "status": "pending"
        }
    finally:
        conn.close()

In [ ]:
class TrackOrder(BaseModel):
    order_id: str = Field(description="The id of the order to track")


def track_order(order_id: str):
    conn = sqlite3.connect(db_name)
    try:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT 
                o.order_id,
                o.customer_name,
                o.customer_email,
                o.customer_phone,
                o.order_date,
                o.order_status,
                p.product_id,
                p.name,
                p.description,
                p.price,
                p.image_url,
                p.link,
                oi.quantity
            FROM orders o
            LEFT JOIN order_items oi ON o.order_id = oi.order_id
            LEFT JOIN products p ON oi.product_id = p.product_id
            WHERE o.order_id = ?
        """, (order_id,))
        rows = cursor.fetchall()
    finally:
        conn.close()

    if not rows:
        return {"error": "Order not found"}

    order_data = {
        "order_id": rows[0][0],
        "customer_name": rows[0][1],
        "customer_email": rows[0][2],
        "customer_phone": rows[0][3],
        "order_date": rows[0][4],
        "order_status": rows[0][5],
        "order_items": []
    }

    for row in rows:
        if row[6] is not None:  # product exists
            order_data["order_items"].append({
                "product_id": row[6],
                "name": row[7],
                "description": row[8],
                "price": row[9],
                "image": row[10],
                "link": row[11],
                "quantity": row[12]
            })

    return order_data

In [ ]:
class UpdateOrder(BaseModel):
    order_id: str = Field(description="The id of the order to update")
    customer_email: Optional[str] = Field(description="The email of the customer")
    customer_phone: Optional[str] = Field(description="The phone number of the customer")

def update_order(order_id: str, customer_email: str, customer_phone: str):
    conn = sqlite3.connect(db_name)
    try:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE orders
            SET customer_email = ?, customer_phone = ?
            WHERE order_id = ?
        """, (customer_email, customer_phone, order_id))
        conn.commit()
    finally:
        conn.close()
    return track_order(order_id)

In [ ]:

class CancelOrder(BaseModel):
    order_id: str = Field(description="The id of the order to cancel")

def cancel_order(order_id: str):
    conn = sqlite3.connect(db_name)
    try:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE orders
            SET order_status = 'cancelled'
            WHERE order_id = ?
        """, (order_id,))
        conn.commit()
    finally:
        conn.close()
    return track_order(order_id)

In [ ]:
class WebSearch(BaseModel):
    query: str = Field(description="The query to search for")
    num_results: Optional[int] = Field(description="The number of results to return")

# web search tool using serp api
def web_search(query: str, num_results: int = 5) -> str:
    api_key = os.getenv("SERPAPI_KEY")
    if not api_key:
        return "Error: SERPAPI_KEY environment variable is not set."

    url = "https://serpapi.com/search.json"
    params = {
        "q": query,
        "engine": "google",
        "api_key": api_key,
        "num": num_results
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()

        if "error" in data:
            return f"Search error: {data['error']}"

        organic = data.get("organic_results", [])
        if not organic:
            return "No search results found."

        parts = []
        for i, r in enumerate(organic, 1):
            title = r.get("title", "")
            link = r.get("link", "")
            snippet = r.get("snippet", "")
            parts.append(f"{i}. {title}\n   {link}\n   {snippet}")

        return "\n\n".join(parts)

    except Exception as e:
        return f"Search failed: {e}"

In [ ]:
openapi_search_tool = {"type": "web_search"}

tools = [
    SearchProducts, 
    CreateOrder, 
    TrackOrder,
    UpdateOrder, 
    CancelOrder, 
    WebSearch,
    FetchKnowledge,
    WebSearch,
    # openapi_search_tool # this doesn't work with all gpt models

]

tools_dict = {
    "SearchProducts": search_products,
    "CreateOrder": create_order,
    "TrackOrder": track_order,
    "UpdateOrder": update_order,
    "CancelOrder": cancel_order,
    "WebSearch": web_search,
    "FetchKnowledge": fetch_knowledge
}

model_with_tools = model.bind_tools(
    tools,
)


In [ ]:
def handle_tool_calls(response):
    responses = []
    for tool_call in response.tool_calls:
        print(tool_call)
        tool_name = tool_call['name']
        tool_input = tool_call['args']
        tool_func = tools_dict.get(tool_name, "Unknown tool")
        if tool_func == "Unknown tool":
            return [
                {
                    'role': 'tool',
                    'content': "Error: Unknown tool",
                    'tool_call_id': tool_call['id']
                }
            ]
        tool_result = tool_func(**tool_input)
        content = tool_result if isinstance(tool_result, str) else json.dumps(tool_result)

        response = {
            'role': 'tool',
            'content': content,
            'tool_call_id': tool_call['id']
        }
        responses.append(response)
    print(responses)
    return responses


In [ ]:
# chat function for gradio interface
def chat(message, history):
    print(message)
    print(history)
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = model_with_tools.invoke(messages)
    while response.tool_calls:
        print('tool-response', response)
        message = response        
        tool_result = handle_tool_calls(response)
        messages.append(message)
        messages.extend(tool_result)
        print("messages", messages)
        response = model_with_tools.invoke(messages)
    yield response.content
                

In [ ]:
# gradio interface

gr.ChatInterface(chat, title="Ecommerce Bot", description="Ask anything about the products and services offered by the store", type='messages').launch()